In [1]:
%load_ext autoreload
%autoreload 2
# set PYTORCH_ENABLE_MPS_FALLBACK=1 in your environment

import os 
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'
import yaml
import torch
import lightning_toolbox as ltb
import lightning
import dypy as dy
import sys
sys.path.append('..')
from ocd.models.permutation.utils import all_permutations
# import sys
# import ocd
# import random
device = 'cpu'
# create a random permutation of n numbers

data_config = yaml.safe_load(
    open("../experiments/intervention/data/chain-3.yml", "r")
)
model_config = yaml.safe_load(open("../experiments/intervention/model.yml", "r"))

n = 30
model_config["init_args"]["model_args"]["in_features"] = n

# setup fixed perm
enforce_perm = torch.randperm(n).tolist()
data_config["init_args"]["dataset_args"]["scm_generator_args"]["graph_generator_args"][
    "enforce_ordering"
] = enforce_perm
model_config["init_args"]["model_args"]["ordering"] = enforce_perm
model_config["init_args"]["model_args"]["scale_transform_s_args"] = dict(post_act_scale=5.0, normalization=None)

In [2]:
from lightning import seed_everything

args = dict(
    # num_hard_samples=100,
    permutation_type="gumbel-topk",
    # permutation_type='soft'
     maximum_basis_size=4,
)
batch_size = 512

model_config["init_args"]["model_args"]["permutation_learner_args"] = args
seed_everything(42)
model_config["init_args"]["model_args"]["permutation_learner_cls"] = "ocd.models.permutation._module_debugging.LearnablePermutation"
model_new = dy.eval(model_config["class_path"])(**model_config["init_args"])
model_config["init_args"]["model_args"][
    "permutation_learner_cls"
] = "ocd.models.permutation.LearnablePermutation"
model_old = dy.eval(model_config["class_path"])(**model_config["init_args"])
model_new.model.permutation_model.gamma = model_old.model.permutation_model.gamma
model_new.model.permutation_model, model_old.model.permutation_model


Global seed set to 42


(LearnablePermutation(num_features=30, permutation_type=gumbel-topk),
 LearnablePermutation(num_features=30))

In [3]:
seed_everything(42)
new_results1 = model_new.model.permutation_model(batch_size)
seed_everything(42)
new_results2 = model_new.model.permutation_model(batch_size)
print((new_results1["soft_perm_mat"] - new_results2["soft_perm_mat"]).abs().max())
print((new_results1["hard_perm_mat"] - new_results2["hard_perm_mat"]).abs().max().max())


Global seed set to 42
Global seed set to 42


tensor(0., grad_fn=<MaxBackward1>)
tensor(0.)


In [4]:
seed_everything(42)
old_results1 = model_old.model.permutation_model(batch_size)
seed_everything(42)
old_results2 = model_old.model.permutation_model(batch_size)
print((old_results1["soft_perm_mat"] - old_results2["soft_perm_mat"]).abs().max())
print((old_results1["hard_perm_mat"] - old_results2["hard_perm_mat"]).abs().max().max())


Global seed set to 42
/Users/vahidzee/.pyenv/versions/3.10.9/envs/deep/lib/python3.10/site-packages/torch/nn/functional.py:1967: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
Global seed set to 42


tensor(0., grad_fn=<MaxBackward1>)
tensor(0.)


In [5]:
print(
    (old_results1["soft_perm_mat"] - new_results1["soft_perm_mat"]).abs().max(),
    new_results1["soft_perm_mat"].shape,
    old_results1["soft_perm_mat"].shape,
)
print(
    (old_results1["hard_perm_mat"] - new_results1["hard_perm_mat"]).abs().max().max(),
    new_results1["hard_perm_mat"].shape,
    old_results1["hard_perm_mat"].shape,
)
print(
    (old_results1['scores'] - new_results1['score_grid']).abs().max().max(), old_results1['scores'].shape, new_results1['score_grid'].shape
)


tensor(0., grad_fn=<MaxBackward1>) torch.Size([30, 30]) torch.Size([30, 30])
tensor(0.) torch.Size([4, 30, 30]) torch.Size([4, 30, 30])
tensor(0., grad_fn=<MaxBackward1>) torch.Size([4]) torch.Size([4])


In [2]:
from lightning import seed_everything

args = dict(
    # num_hard_samples=100,
    permutation_type="hard",
    # permutation_type='soft'
)
batch_size = 512

model_config["init_args"]["model_args"]["permutation_learner_args"] = args
seed_everything(42)
model_config["init_args"]["model_args"]["permutation_learner_cls"] = "ocd.models.permutation._module_debugging.LearnablePermutation"
model_new = dy.eval(model_config["class_path"])(**model_config["init_args"])
model_config["init_args"]["model_args"][
    "permutation_learner_cls"
] = "ocd.models.permutation.LearnablePermutation"
model_old = dy.eval(model_config["class_path"])(**model_config["init_args"])
model_new.model.permutation_model.gamma = model_old.model.permutation_model.gamma
model_new.model.permutation_model, model_old.model.permutation_model


Global seed set to 42


(LearnablePermutation(num_features=30, permutation_type=hard),
 LearnablePermutation(num_features=30))

In [3]:
seed_everything(42)
new_results1 = model_new.model.permutation_model(batch_size)
seed_everything(42)
new_results2 = model_new.model.permutation_model(batch_size)
print((new_results1["perm_mat"] - new_results2["perm_mat"]).abs().max())


Global seed set to 42
Global seed set to 42


tensor(0.)


In [4]:
seed_everything(42)
old_results1 = model_old.model.permutation_model(batch_size)
seed_everything(42)
old_results2 = model_old.model.permutation_model(batch_size)
print((old_results1["perm_mat"] - old_results2["perm_mat"]).abs().max(), "a")


Global seed set to 42
Global seed set to 42


tensor(0.) a


In [5]:
print(
    (old_results1["perm_mat"] - new_results1["perm_mat"]).abs().max(),
    new_results1["perm_mat"].shape,
    old_results1["perm_mat"].shape,
)


tensor(0.) torch.Size([512, 30, 30]) torch.Size([512, 30, 30])


In [2]:
from lightning import seed_everything

args = dict(
    # num_hard_samples=100,
    permutation_type="soft",
    # permutation_type='soft'
)
batch_size = 512

model_config["init_args"]["model_args"]["permutation_learner_args"] = args
seed_everything(42)
model_config["init_args"]["model_args"]["permutation_learner_cls"] = "ocd.models.permutation._module_debugging.LearnablePermutation"
model_new = dy.eval(model_config["class_path"])(**model_config["init_args"])
model_config["init_args"]["model_args"][
    "permutation_learner_cls"
] = "ocd.models.permutation.LearnablePermutation"
model_old = dy.eval(model_config["class_path"])(**model_config["init_args"])
model_new.model.permutation_model.gamma = model_old.model.permutation_model.gamma
model_new.model.permutation_model, model_old.model.permutation_model


Global seed set to 42


(LearnablePermutation(num_features=30), LearnablePermutation(num_features=30))

In [3]:
seed_everything(42)
new_results1 = model_new.model.permutation_model(batch_size)
seed_everything(42)
new_results2 = model_new.model.permutation_model(batch_size)
print((new_results1["perm_mat"] - new_results2["perm_mat"]).abs().max())


Global seed set to 42
Global seed set to 42


tensor(0., grad_fn=<MaxBackward1>)


In [4]:
seed_everything(42)
old_results1 = model_old.model.permutation_model(batch_size)
seed_everything(42)
old_results2 = model_old.model.permutation_model(batch_size)
print((old_results1["perm_mat"] - old_results2["perm_mat"]).abs().max())


Global seed set to 42
Global seed set to 42


tensor(0., grad_fn=<MaxBackward1>)


In [5]:
print(
    (old_results1["perm_mat"] - new_results1["perm_mat"]).abs().max(),
    new_results1["perm_mat"].shape,
    old_results1["perm_mat"].shape,
)


tensor(0., grad_fn=<MaxBackward1>) torch.Size([512, 30, 30]) torch.Size([512, 30, 30])


In [2]:
from lightning import seed_everything

args = dict(
    # num_hard_samples=100,
    permutation_type="hybrid-straight-through",
    hard_from_softs=True,
)
batch_size = 512

model_config["init_args"]["model_args"]["permutation_learner_args"] = args
seed_everything(42)
model_config["init_args"]["model_args"]["permutation_learner_cls"] = "ocd.models.permutation._module_debugging.LearnablePermutation"
model_new = dy.eval(model_config["class_path"])(**model_config["init_args"])
model_config["init_args"]["model_args"][
    "permutation_learner_cls"
] = "ocd.models.permutation.LearnablePermutation"
args = dict(
    # num_hard_samples=100,
    permutation_type="straight-through",
)
batch_size = 512

model_config["init_args"]["model_args"]["permutation_learner_args"] = args
model_old = dy.eval(model_config["class_path"])(**model_config["init_args"])
model_new.model.permutation_model.gamma = model_old.model.permutation_model.gamma
model_new.model.permutation_model, model_old.model.permutation_model


Global seed set to 42


(LearnablePermutation(num_features=30, permutation_type=hybrid-straight-through),
 LearnablePermutation(num_features=30))

In [3]:
seed_everything(42)
new_results1 = model_new.model.permutation_model(batch_size)
seed_everything(42)
new_results2 = model_new.model.permutation_model(batch_size)
print((new_results1["perm_mat"] - new_results2["perm_mat"]).abs().max())


Global seed set to 42
Global seed set to 42


tensor(0., grad_fn=<MaxBackward1>)


In [4]:
seed_everything(42)
old_results1 = model_old.model.permutation_model(batch_size)
seed_everything(42)
old_results2 = model_old.model.permutation_model(batch_size)
print((old_results1["perm_mat"] - old_results2["perm_mat"]).abs().max())


Global seed set to 42
Global seed set to 42


tensor(0., grad_fn=<MaxBackward1>)


In [5]:
print(
    (old_results1["perm_mat"] - new_results1["perm_mat"]).abs().max(),
    new_results1["perm_mat"].shape,
    old_results1["perm_mat"].shape,
)


tensor(0., grad_fn=<MaxBackward1>) torch.Size([512, 30, 30]) torch.Size([512, 30, 30])


In [ ]:
from ocd.models.permutation.utils import sinkhorn, sample_gumbel_noise, hungarian, listperm2matperm

num_features = 10
num_samples = 10000
num_iters = 50
tau = 0.1
gamma = torch.randn(num_features, num_features)
f = lambda x: -torch.nn.functional.logsigmoid(x)
# f = lambda x: x

noise = sample_gumbel_noise(num_samples, num_features, num_features, device=device) * 1
soft_noise_in = sinkhorn(f((gamma) + noise) / tau, num_iters=num_iters)
soft_noise_out = sinkhorn(f((gamma)) / tau, num_iters=num_iters) + noise / tau

# generate soft permutaiton
hard_plain = hungarian((f(gamma) + noise) / tau)
hard_soft_in = hungarian(soft_noise_in)
hard_soft_in_1 = hungarian(sinkhorn(f((gamma) + noise) / tau, num_iters=1))
hard_soft_out = hungarian(soft_noise_out)

torch.unique(hard_plain, dim=0).shape, torch.unique(hard_soft_in, dim=0).shape, torch.unique(
    hard_soft_out, dim=0
).shape, torch.unique(hard_soft_in_1, dim=0).shape
